In [2]:
from folium import Map, Circle
import pandas


In [7]:
#arbres_raw = pandas.read_csv(
#    "https://data.metropolegrenoble.fr/sites/default/files/dataset/2025/02/28/76124734-6258-4184-a964-1af2e4f68ce9/arbres_grenoble_epsg4326.csv"
#)
#arbres_raw_bis = pandas.read_csv(
#    "https://data.metropolegrenoble.fr/d4c/api/records/2.0/downloadfile/format=csv&resource_id=05566402-cd0d-4c9b-90b6-4a90e4629e03&use_labels_for_header=true&user_defined_fields=true"
#)
arbres_raw = pandas.read_csv("arbres_de_grenoble.csv")

In [ ]:
grenoble = (45.18, 5.73)
m = Map(grenoble, zoom_start=12, prefer_canvas=True)
max_trees = 100000
radius=1.5
for i in range(len(arbres_raw)):
    coordonnées = str.split(arbres_raw.geo_point_2d[i], ",")
    if "" in coordonnées :# or arbres_raw.genre_bota[i] != "Sequoiadendron":
        continue
    x = float(coordonnées[0])
    y = float(coordonnées[1])
    nom_commun = arbres_raw.nom_bota[i]
    résumé = arbres_raw.resume[i]
    Circle(
        location=[x, y],
        radius=radius,
        color="green",
        opacity=1,
        fill_color="green",
        fill_opacity=0.5,
        popup=f'{nom_commun}\n\n{résumé}',
        tooltip=nom_commun
    ).add_to(m)
    if i > max_trees : break
m

In [5]:
def has_dupes(l: list) -> bool:
    return len(l) != len(set(l))


def has_diff_couple(l1, l2):
    d: dict = {}
    for i in range(len(l1)):
        if l1[i] not in d:
            d[l1[i]] = l2[i]
        elif d[l1[i]] != l2[i]:
            return True
    return False

In [6]:
has_dupes(arbres_raw.bien_reference)


False

In [7]:
#has_diff_couple([arbres_raw.genre_bota[i] + arbres_raw.espece[i] for i in range(len(arbres_raw.nom))] , 
#                arbres_raw.genre_bota)

In [8]:
a = ["a", "b"]
b = ["c", "d"]
[a[i]+b[i] for i in range(len(a))]

['ac', 'bd']

In [9]:
genre="Alnus"
s = {1}
for i in range(len(arbres_raw)):
    if arbres_raw.genre_bota[i] == genre:
        s = s | {arbres_raw.espece[i]}
for i in s:
    print(i)

glutinosa
1
cordata
incana
acerifolia
lamarckii
ornus
spaethii
nan
americana
cerris
platyphyllos


In [10]:
genres = set(arbres_raw.genre_bota)
genres.remove('inconnu')
genres


{'Abies',
 'Acacia',
 'Acer',
 'Aesculus',
 'Ailanthus',
 'Albizia',
 'Alnus',
 'Amélanchier',
 'Araucaria',
 'Betula',
 'Broussonetia',
 'Buxus',
 'Calocedrus',
 'Carpinus',
 'Carya',
 'Castanea',
 'Catalpa',
 'Cedrela',
 'Cedrus',
 'Celtis',
 'Cephalotaxus',
 'Cercidiphyllum',
 'Cercis',
 'Chamaecyparis',
 'Chimonanthus',
 'Chionanthus',
 'Chitalpa',
 'Cladastris',
 'Clerodendron',
 'Cornus',
 'Corylus',
 'Cotinus',
 'Crataegus',
 'Cryptomeria',
 'Cupressocyparis',
 'Cupressus',
 'Cydonia',
 'Davidia',
 'Diospyros',
 'Eleagnus',
 'Eriobotrya ',
 'Eucalyptus',
 'Fagus',
 'Ficus',
 'Fontanesia',
 'Fraxinus',
 'Ginkgo',
 'Gleditsia',
 'Gymnocladus',
 'Halesia',
 'Hibiscus',
 'Hovenia',
 'Idesia',
 'Ilex',
 'Juglans',
 'Juniperus',
 'Koelreuteria',
 'Laburnum',
 'Lagerstroemia',
 'Larix',
 'Laurus',
 'Libocedrus',
 'Ligustrum',
 'Liquidambar',
 'Liriodendron',
 'Lonicera',
 'Maclura',
 'Magnolia',
 'Malus',
 'Melia',
 'Mespilus',
 'Metasequoia',
 'Morus',
 'Nyssa',
 'Olea',
 'Osmanthus',

In [11]:
import wikipedia
import requests
import json

wikipedia.set_lang("fr")

def get_wiki_main_image(title):
    url = 'https://fr.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : title
    }
    response = requests.get(url, data)
    json_data = json.loads(response.text)
    return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'

In [28]:
genre_title = {}
genre_summary = {}
data = []
i = 0
for genre in genres:
    i+=1
    if i>10:
        pass
    title = ""
    summary = ""
    lien_image = ""

    try:
        page = wikipedia.page(genre, auto_suggest=False)
        title = page.title
        summary = page.summary
    except:
        title = genre
        
    try:
        lien_image = get_wiki_main_image(title)
    except:
        pass

    genre_title[genre] = title
    genre_summary[summary] = summary

    print(f"{title} ({genre})\n   {summary}\n")
    data.append({
        "genre_bota": genre,
        "nom_bota": title,
        "resume": summary,
        "lien_image": ""
    })

import csv

keys = data[0].keys()

with open('treesbis.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

Ficus (Ficus)
   Ficus est un genre de plantes à fleurs de la famille des Moraceae, représenté par des arbres, des arbustes ou des lianes. Avec plus de 750 espèces connues il s'agit, avec Dorstenia, d'un des principaux genres de sa famille. Ce sont principalement des plantes tropicales dont toutes ont en commun de produire des inflorescences et infrutescences particulières, les sycones ou figues.
Ficus était  en latin le nom commun du figuier, l'arbre cultivé dans le bassin méditerranéen pour ses fruits comestibles, les figues.
Par extension, tous les représentants du genre Ficus peuvent être appelés « figuiers », mais dans l'usage courant du français de nombreuses espèces sont connues surtout sous leur nom scientifique, notamment en horticulture (Ficus benjamina, Ficus lyrata…) tandis que d'autres possèdent des appellations populaires ou vernaculaires spécifiques : Caoutchouc (Ficus elastica), Banyan (Ficus benghalensis), Affouches (Ficus densifolia, Ficus reflexa…), etc.



Fagus (Fa

/home/maxlamenace/Documents/Enseignement/Documents/3 - TNSI/Bases de données/3 - Projet Arbres/venv_bdd_arbres/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/maxlamenace/Documents/Enseignement/Documents/3 - TNSI/Bases de données/3 - Projet Arbres/venv_bdd_arbres/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Cryptomeria (Cryptomeria)
   

Olea (Olea)
   Le terme Olea désigne un taxon, le genre Olea, c'est-à-dire un ensemble de plantes ayant des caractéristiques communes. Trois sous-genres (subg., niveaux de taxonomie) ont été créés à la suite des travaux de révision menés par le botaniste P.S. Green, Il aboutit à reconnaître et à regrouper 33 espèces et neuf sous-espèces parmi les nombreuses synonymies qui se sont établies en trois siècles de botanique et d'explorations du monde.
Trois nouveaux taxons ont été reconnus : le sous-genre Tetrapilus, le sous-genre Paniculatae et Olea woodiana subsp. disjuncta. La nouvelle classification des taxons, dont le sous-genre Olea, s'établit comme suit :

Sous-genre Olea, comprenant 2 sections :
Section Olea, c'est-à-dire un groupe d'espèces végétales appartenant au sous-genre Olea, comprenant 6 sous-espèces,
Section Ligustroides,
subg. Tetrapilus,
subg. Paniculatae.
Cette classification peut sembler complexe, elle est le fruit du travail du botaniste P

In [ ]:
wikipedia.summary("Chêne", sentences=1)

In [ ]:
import wikipedia

wikipedia.set_lang("fr")
print(genres)
pages = {}
for genre in genres:
    print(genre)
    search_list = wikipedia.search(genre)
    if len(search_list) == 0:
        search_list = wikipedia.search(wikipedia.suggest(genre))
    print(search_list[0])
    try:
        search = wikipedia.page(search_list[0], auto_suggest=False).title
    except wikipedia.DisambiguationError as e:
        print("DISAMBIGUATION ERROR - PLEASE CHOOSE AN OPTION")
        for i in range(len(e.options)):
            print(f"{i+1} - {e.options[i]}")
        choice = input("Which option ?")
        search = wikipedia.page(e.options[choice-1]).title
    summary = ""
    try:
        summary = wikipedia.summary(search, sentences=1)
    except:
        ...
    print(f"{search.upper()} ({genre})\n   {summary}\n")

In [26]:
page = wikipedia.page("ficus", auto_suggest=False)
print(page)
title = page.title
print(title)
summary = page.summary
print(summary)

<WikipediaPage 'Ficus'>
Ficus
Ficus est un genre de plantes à fleurs de la famille des Moraceae, représenté par des arbres, des arbustes ou des lianes. Avec plus de 750 espèces connues il s'agit, avec Dorstenia, d'un des principaux genres de sa famille. Ce sont principalement des plantes tropicales dont toutes ont en commun de produire des inflorescences et infrutescences particulières, les sycones ou figues.
Ficus était  en latin le nom commun du figuier, l'arbre cultivé dans le bassin méditerranéen pour ses fruits comestibles, les figues.
Par extension, tous les représentants du genre Ficus peuvent être appelés « figuiers », mais dans l'usage courant du français de nombreuses espèces sont connues surtout sous leur nom scientifique, notamment en horticulture (Ficus benjamina, Ficus lyrata…) tandis que d'autres possèdent des appellations populaires ou vernaculaires spécifiques : Caoutchouc (Ficus elastica), Banyan (Ficus benghalensis), Affouches (Ficus densifolia, Ficus reflexa…), etc.

In [37]:
import pandas as pd

b = pd.read_csv("arbres_grenoble_epsg4326.csv")
a = pd.read_csv("treesbis.csv")


In [38]:
b

,elem_point_id,code,nom,genre,genre_desc,categorie,categorie_desc,sous_categorie,sous_categorie_desc,code_parent,...,diametrearbre,causeabattage,collectivite,stationmetro,hauteurarbre,portarbre,structure,forme,typenature,geo_point_2d
0,34835,ESP36318,AR36318,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP237,...,NaN,NaN,Grenoble Alpes Métropole,NaN,NaN,NaN,Métropole de Grenoble,NaN,NaN,"45.167098421885385,5.740131798404362"
1,34836,ESP36319,AR36319,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP237,...,NaN,NaN,Grenoble Alpes Métropole,NaN,NaN,NaN,Métropole de Grenoble,NaN,NaN,"45.16710682038838,5.740199792549315"
2,34837,ESP36320,AR36320,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP237,...,NaN,NaN,Grenoble Alpes Métropole,NaN,NaN,NaN,Métropole de Grenoble,NaN,NaN,"45.167115270975415,5.740265608803635"
3,34838,ESP36321,AR36321,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP237,...,NaN,NaN,Grenoble Alpes Métropole,NaN,NaN,NaN,Métropole de Grenoble,NaN,NaN,"45.16712715617142,5.740349032579623"
4,34839,ESP36322,AR36322,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP237,...,NaN,NaN,Grenoble Alpes Métropole,NaN,NaN,NaN,Métropole de Grenoble,NaN,NaN,"45.167144245872265,5.740471081600832"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31665,44741,ESP40744,AR40744,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP479,...,NaN,NaN,Grenoble Alpes Métropole,NaN,Moins de 10 m,NaN,Métropole de Grenoble,Tige,NaN,"45.19130940286065,5.721605325148501"
31666,44742,ESP40745,AR40745,VEG,VEGETATION,ESP01,Arbre,ESP151,Arbre de voirie,ESP479,...,NaN,NaN,Grenoble Alpes Métropole,NaN,Moins de 10 m,NaN,Métropole de Grenoble,Tige,NaN,"45.19135228905578,5.721902795690901"
31667,44841,ESP40796,AR40796,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45.18358251458179,5.714244367296723"
31668,44842,ESP40797,AR40797,VEG,VEGETATION,ESP01,Arbre,ESP174,Arbre d'espaces ouverts,ESP273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45.183576237082896,5.714287511308287"


In [40]:
merged = b.merge(a, on='genre_bota')
merged.to_csv("arbres_de_grenoble.csv", index=False)